# Creando variables Campañas

Código para validar cruces y crear la variable respuesta

In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas_profiling
import os
from datetime import datetime, timedelta
import dateutil.relativedelta
import gc

pd.set_option('display.max_columns',None)

In [2]:

def func_activa(base,date,n_mes,nom_col):
    if len(base)>0:
        date_fin = date + relativedelta(months=n_mes)

        data_act_temp = data_act.loc[(data_act['mes_activacion']>=date) & (data_act['mes_activacion']<=date_fin),['cedula','ID_CLIENTE','Numero_cuenta','fecha_activacion','mes_activacion']].drop_duplicates()
        ids_mes = data_act_temp['cedula'].unique()
        
        base.loc[base['cedula'].isin(ids_mes),'venta'] = 1    
        base.loc[base['cedula'].isnull(),'venta'] = 0
        
        base.loc[:,nom_col] = 1   
        base = pd.merge(base,data_act_temp,on=['cedula','ID_CLIENTE'],how='left')
    
    return base

# Cargando bases de campañas

## Oficinas

In [3]:
path_file='s3://adl-refined-dev-popular/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv'

data = pd.read_csv(path_file, sep = '|', dtype={'ID_CLIENTE':str})
data = data.drop_duplicates()
print(data.shape)
data.head()

(9193, 3)


,NUM_DOC,ID_CLIENTE,FECHA
0,79711820,248852317364653702,04/01/2019
1,20896135,724752296893347302,04/01/2019
2,41527303,733752313515338501,04/01/2019
3,79491343,148752314504163701,04/01/2019
4,39702994,746152314858805501,04/01/2019


In [4]:
path_file1='s3://adl-refined-dev-popular/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv'

data1 = pd.read_csv(path_file1, sep = '|', dtype={'ID_CLIENTE':str})
data1 = data1.drop_duplicates()
print(data1.shape)
data1.head()

(10668, 3)


,NUM_DOC,ID_CLIENTE,FECHA
0,21374060,687052296965327502,01/11/2019
1,54254979,996352303010557001,01/11/2019
2,17136481,758052296415581701,01/11/2019
3,8389506,584352312393754001,01/11/2019
4,41369835,708954239890969602,01/11/2019


In [5]:
data1 = pd.concat([data,data1], ignore_index=True)

del data

In [6]:
## tipo campaña
data1['TIPO CAMPAÑA'] = 'Preaprobados'
data1['tipo_camp'] = 'of-preaprobados'

In [7]:
# Casting variables
data1['FECHA'] = pd.to_datetime(data1['FECHA'],format='%d/%m/%Y')
data1['mes_campaña'] = pd.to_datetime(np.where(data1['FECHA'].dt.day>=28, 
                                                              data1['FECHA']+timedelta(days=10),
                                                              data1['FECHA']),format='%Y-%m-%d')
data1['mes_campaña2'] = pd.to_datetime(data1['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
data1['mes_campaña'] = pd.to_datetime(data1['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')

In [8]:
data1.columns

Index(['NUM_DOC', 'ID_CLIENTE', 'FECHA', 'TIPO CAMPAÑA', 'tipo_camp',
       'mes_campaña', 'mes_campaña2'],
      dtype='object')

## Atento

In [9]:
path_file2='s3://adl-refined-dev-popular/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv'

data2 = pd.read_csv(path_file2, sep = '|', encoding='ISO8859-1',quoting=1, dtype={'ID_CLIENTE':str})
data2 = data2.drop_duplicates()
print(data2.shape)
data2.head()

(77455, 6)


,TIP_DOC,NUM_DOC,ID_CLIENTE,CUPO_APROBADO,FECHA,TIPO CAMPAÑA
0,1.0,64543703,779452303196638001,NaN,01/02/2019,Perfilados Activos
1,1.0,1013652262,767952313507880801,NaN,01/02/2019,Perfilados Activos
2,1.0,80188542,699952305504125601,NaN,01/02/2019,Perfilados Activos
3,1.0,1022989795,203652310816127502,NaN,01/02/2019,Perfilados Activos
4,1.0,79407899,379352315012448502,NaN,01/02/2019,Perfilados Activos


In [10]:
# Casting variables
data2['tipo_camp'] = np.where(data2['TIPO CAMPAÑA'] == 'Preaprobados','at-preaprobados','at-perfilados')
data2['FECHA'] = pd.to_datetime(data2['FECHA'],format='%d/%m/%Y')
data2['mes_campaña'] = pd.to_datetime(data2['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')

In [11]:
data2.columns

Index(['TIP_DOC', 'NUM_DOC', 'ID_CLIENTE', ' CUPO_APROBADO ', 'FECHA',
       'TIPO CAMPAÑA', 'tipo_camp', 'mes_campaña'],
      dtype='object')

## Activaciones

In [12]:
path_act='s3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-nueva/productos_activos-tarjeta-credito-activadas_masterfile_H20180101-20200327.csv'

data_act = pd.read_csv(path_act, sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str})
data_act = data_act.drop_duplicates()

print(data_act.shape)
data_act

(40411, 6)


,Numero_cuenta,Numero_tarjeta,Identificacion,Tipo_identificacion,fecha_activacion,ID_CLIENTE
0,5392386030737574936,539238******7338,72376,2,29/01/2018,489152311997977401
1,4205594037122388692,420559******1701,72376,2,29/01/2018,489152311997977401
2,5391686024870915922,539168******9961,131658,2,29/10/2019,851557066763798702
3,4544054304149486544,454405******9846,131658,2,29/10/2019,851557066763798702
4,4205594035742557191,420559******1788,132796,2,27/09/2019,139052309749044702
...,...,...,...,...,...,...
40417,4544054306697232527,454405******2206,1234091635,1,23/08/2018,286055720866429601
40418,4066944507830597993,406694******0828,1234188867,1,18/01/2019,106155721625491802
40419,4506584008016663139,450658******3429,1234192603,1,19/03/2020,NaN
40420,4506584002386442927,450658******1032,1234641193,1,19/02/2020,NaN


In [13]:
# Casting variables
data_act['fecha_activacion'] = pd.to_datetime(data_act['fecha_activacion'],format='%d/%m/%Y')
data_act['mes_activacion'] = pd.to_datetime(data_act['fecha_activacion'].dt.strftime('%Y-%m'),format='%Y-%m')

In [14]:
data_act = data_act[data_act['ID_CLIENTE'].notnull()]

### Uniendo bases 

In [15]:
cols1 = ['NUM_DOC', 'ID_CLIENTE', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp']
cols2 = ['NUM_DOC', 'ID_CLIENTE', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp', ' CUPO_APROBADO ']

In [16]:
data_camp = pd.concat([data1[cols1], data2[cols2]], ignore_index=True)[cols2]
print(data_camp.shape)
data_camp 

(97316, 7)


/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,NUM_DOC,ID_CLIENTE,FECHA,mes_campaña,TIPO CAMPAÑA,tipo_camp,CUPO_APROBADO
0,79711820,248852317364653702,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,NaN
1,20896135,724752296893347302,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,NaN
2,41527303,733752313515338501,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,NaN
3,79491343,148752314504163701,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,NaN
4,39702994,746152314858805501,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,NaN
...,...,...,...,...,...,...,...
97311,17007499,132452296355094801,2020-03-01,2020-03-01,Preaprobados,at-preaprobados,19.600.000
97312,17112493,246955266243335502,2020-03-01,2020-03-01,Preaprobados,at-preaprobados,13.500.000
97313,3362730,102652305802784101,2020-03-01,2020-03-01,Preaprobados,at-preaprobados,9.000.000
97314,3430980,339352294618484201,2020-03-01,2020-03-01,Preaprobados,at-preaprobados,2.400.000


In [17]:
## Renombrando variables
data_act = data_act.rename(columns={'Identificacion':'cedula'})
data_camp = data_camp.rename(columns={'NUM_DOC':'cedula'})

In [18]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

str_dates = ['2019-01', '2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
          '2019-12','2020-01','2020-02','2020-03']

n_mes = 3 # numero de meses

for ix, date in enumerate(str_dates):
    print('Fecha:',date)
    date = pd.to_datetime(date,format='%Y-%m')
    nom_col = 'camp_'+date.strftime('%Y%m')
    
    base = data_camp[data_camp['mes_campaña']==date]

    if ix == 0:
        base_final = func_activa(base,date,n_mes,nom_col)
        del base
    else:
        base_temp = func_activa(base,date,n_mes,nom_col)
        base_final = pd.concat([base_final,base_temp],ignore_index=True)
        
        del base, base_temp
        

/home/jupyter/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/jupyter/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort b

Fecha: 2019-01
Fecha: 2019-02
Fecha: 2019-03
Fecha: 2019-04
Fecha: 2019-05
Fecha: 2019-06
Fecha: 2019-07
Fecha: 2019-08
Fecha: 2019-09
Fecha: 2019-10
Fecha: 2019-11
Fecha: 2019-12
Fecha: 2020-01
Fecha: 2020-02
Fecha: 2020-03


In [19]:
## Casting variables
base_final = base_final[base_final['ID_CLIENTE'].notnull()]

base_final.rename(columns={' CUPO_APROBADO ':'CUPO_APROBADO'}, inplace=True)
base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].str.strip()
base_final['CUPO_APROBADO'] = [str(x).replace('.','') for x in base_final['CUPO_APROBADO']]
base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].astype('float')

cols = ['camp_201901','camp_201902','camp_201903','camp_201904','camp_201905','camp_201906',
            'camp_201907','camp_201908','camp_201909','camp_201910','camp_201911','camp_201912',
            'camp_202001','camp_202002','camp_202003','venta']
                                                                              
base_final.loc[:,cols] = base_final[cols].fillna(0)

In [20]:
# Casting variables
base_final['fecha_activacion'] = pd.to_datetime(base_final['fecha_activacion'], format='%Y-%m%d')
base_final['mes_activacion'] = pd.to_datetime(base_final['mes_activacion'], format='%Y-%m%d')


base_final

,CUPO_APROBADO,FECHA,ID_CLIENTE,Numero_cuenta,TIPO CAMPAÑA,camp_201901,camp_201902,camp_201903,camp_201904,camp_201905,camp_201906,camp_201907,camp_201908,camp_201909,camp_201910,camp_201911,camp_201912,camp_202001,camp_202002,camp_202003,cedula,fecha_activacion,mes_activacion,mes_campaña,tipo_camp,venta
0,NaN,2019-01-04,248852317364653702,NaN,Preaprobados,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79711820,NaT,NaT,2019-01-01,of-preaprobados,0.0
1,NaN,2019-01-04,724752296893347302,5361706003729575037,Preaprobados,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20896135,2019-02-05,2019-02-01,2019-01-01,of-preaprobados,1.0
2,NaN,2019-01-04,724752296893347302,4506584002622716431,Preaprobados,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20896135,2019-02-05,2019-02-01,2019-01-01,of-preaprobados,1.0
3,NaN,2019-01-04,733752313515338501,NaN,Preaprobados,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41527303,NaT,NaT,2019-01-01,of-preaprobados,0.0
4,NaN,2019-01-04,148752314504163701,NaN,Preaprobados,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79491343,NaT,NaT,2019-01-01,of-preaprobados,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98543,19600000.0,2020-03-01,132452296355094801,NaN,Preaprobados,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17007499,NaT,NaT,2020-03-01,at-preaprobados,0.0
98544,13500000.0,2020-03-01,246955266243335502,NaN,Preaprobados,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17112493,NaT,NaT,2020-03-01,at-preaprobados,0.0
98545,9000000.0,2020-03-01,102652305802784101,NaN,Preaprobados,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3362730,NaT,NaT,2020-03-01,at-preaprobados,0.0
98546,2400000.0,2020-03-01,339352294618484201,NaN,Preaprobados,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3430980,NaT,NaT,2020-03-01,at-preaprobados,0.0


In [21]:
# Número de veces que ha sido incluido en la campaña últimos x meses
# Número de meses desde que fue incluido en la campaña de adquisición por última vez
# Número de activaciones de TC trad últimos x meses
# Meses desde la última activación
# Número de veces que ha sido incluido en la campaña y no aceptado
# Si ya se ha aceptado anteriormente la oferta


In [22]:
base_final['mes_campaña'].value_counts().sort_index()

2019-01-01      478
2019-02-01     4888
2019-03-01     4483
2019-04-01     6159
2019-05-01     6509
2019-06-01     5848
2019-07-01     5925
2019-08-01     7118
2019-09-01     6663
2019-10-01     6518
2019-11-01     5883
2019-12-01     4332
2020-01-01     7404
2020-02-01     8363
2020-03-01    10133
Name: mes_campaña, dtype: int64

In [23]:
base_final.columns

Index(['CUPO_APROBADO', 'FECHA', 'ID_CLIENTE', 'Numero_cuenta', 'TIPO CAMPAÑA',
       'camp_201901', 'camp_201902', 'camp_201903', 'camp_201904',
       'camp_201905', 'camp_201906', 'camp_201907', 'camp_201908',
       'camp_201909', 'camp_201910', 'camp_201911', 'camp_201912',
       'camp_202001', 'camp_202002', 'camp_202003', 'cedula',
       'fecha_activacion', 'mes_activacion', 'mes_campaña', 'tipo_camp',
       'venta'],
      dtype='object')

In [24]:
base_final.dtypes

CUPO_APROBADO              float64
FECHA               datetime64[ns]
ID_CLIENTE                  object
Numero_cuenta               object
TIPO CAMPAÑA                object
camp_201901                float64
camp_201902                float64
camp_201903                float64
camp_201904                float64
camp_201905                float64
camp_201906                float64
camp_201907                float64
camp_201908                float64
camp_201909                float64
camp_201910                float64
camp_201911                float64
camp_201912                float64
camp_202001                float64
camp_202002                float64
camp_202003                float64
cedula                       int64
fecha_activacion    datetime64[ns]
mes_activacion      datetime64[ns]
mes_campaña         datetime64[ns]
tipo_camp                   object
venta                      float64
dtype: object

In [25]:
## Crear variable meses
def func_var_camp_act(gp_temp, ids):
    
    #producing the first dataframe
    base_out = pd.DataFrame(index=ids) # this is different from the actual prod code
    base_out['mes_campaña'] = date.strftime('%Y-%m-%d')
    
    ## Calculando días primer uso
    base_out['mes_ult_camp'] =  gp_temp['mes_campaña'].max()
    base_out['mes_ult_act'] =  gp_temp['fecha_activacion'].max()
    base_out.loc[base_out['mes_ult_camp'].notnull(),'num_meses_ult_camp'] = (date - base_out.loc[base_out['mes_ult_camp'].notnull(),'mes_ult_camp'])/pd.offsets.Day(30)
    base_out.loc[base_out['mes_ult_act'].notnull(),'num_meses_ult_actv'] = (date - base_out.loc[base_out['mes_ult_act'].notnull(),'mes_ult_act'])/pd.offsets.Day(30)
    
    base_out['num_camp_ult_meses'] = gp_temp['mes_campaña'].count()
    base_out['num_act_utl_meses'] = gp_temp['venta'].sum()
    base_out['num_no_aceptado'] = gp_temp['venta'].value_counts().unstack()[0].fillna(0).astype(int)
    base_out['acep_oferta_prev'] = gp_temp['venta'].max()
    
    return base_out

In [26]:
str_dates = ['2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
          '2019-12','2020-01','2020-02','2020-03']


for ix, dates in enumerate(str_dates):
    print('Periodo:', dates)
    date = pd.to_datetime(dates,format='%Y-%m')
    
    ids = list(base_final.loc[base_final['mes_campaña']==date,'ID_CLIENTE'].unique())
    gp_temp = base_final[base_final['mes_campaña']<date].groupby('ID_CLIENTE')
    gp_temp_ap = base_final[(base_final['mes_campaña']<date)&(base_final['CUPO_APROBADO'].notnull())].groupby('ID_CLIENTE')
    
    if ix == 0:
        tabla_final = func_var_camp_act(gp_temp, ids)
        
        if len(gp_temp_ap)>0:
            tabla_final['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
    
        tabla_final.index.rename('id_cliente', inplace=True)
        tabla_final.reset_index(inplace = True)
    else:
        base_temp = func_var_camp_act(gp_temp, ids)
        
        if len(gp_temp_ap)>0:
            base_temp['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
    
        base_temp.index.rename('id_cliente', inplace=True)
        base_temp.reset_index(inplace = True)
        
        tabla_final = pd.concat([tabla_final,base_temp], ignore_index=True)
        
        del base_temp
    del gp_temp, date

## Cast index        
tabla_final['id_cliente'] = tabla_final['id_cliente'].astype('int')    

Periodo: 2019-02
Periodo: 2019-03
Periodo: 2019-04
Periodo: 2019-05


/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Periodo: 2019-06
Periodo: 2019-07
Periodo: 2019-08
Periodo: 2019-09
Periodo: 2019-10
Periodo: 2019-11
Periodo: 2019-12
Periodo: 2020-01
Periodo: 2020-02
Periodo: 2020-03


In [27]:
tabla_final= tabla_final[['id_cliente', 'mes_campaña','acep_oferta_prev', 'mes_ult_act',
       'mes_ult_camp', 'num_act_utl_meses', 'num_camp_ult_meses',
       'num_meses_ult_actv', 'num_meses_ult_camp', 'num_no_aceptado',
       'prom_cupo_aprob']]

In [28]:
tabla_final.count()

id_cliente            88969
mes_campaña           88969
acep_oferta_prev      18456
mes_ult_act              59
mes_ult_camp          18456
num_act_utl_meses     18456
num_camp_ult_meses    18456
num_meses_ult_actv       59
num_meses_ult_camp    18456
num_no_aceptado       18456
prom_cupo_aprob        7535
dtype: int64

In [29]:
tabla_final

,id_cliente,mes_campaña,acep_oferta_prev,mes_ult_act,mes_ult_camp,num_act_utl_meses,num_camp_ult_meses,num_meses_ult_actv,num_meses_ult_camp,num_no_aceptado,prom_cupo_aprob
0,605152297523531501,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,309252297762338201,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,365655266241757701,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,227552315277170601,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,833752296410065002,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
88964,132452296355094801,2020-03-01,0.0,NaT,2019-11-01,0.0,3.0,NaN,4.033333,3.0,19600000.0
88965,246955266243335502,2020-03-01,0.0,NaT,2019-11-01,0.0,2.0,NaN,4.033333,2.0,12000000.0
88966,102652305802784101,2020-03-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
88967,339352294618484201,2020-03-01,0.0,NaT,2019-09-01,0.0,1.0,NaN,6.066667,1.0,NaN


In [31]:
tabla_final.describe()

,id_cliente,acep_oferta_prev,num_act_utl_meses,num_camp_ult_meses,num_meses_ult_actv,num_meses_ult_camp,num_no_aceptado,prom_cupo_aprob
count,8.896900e+04,18456.000000,18456.000000,18456.000000,59.000000,18456.000000,18456.000000,7.535000e+03
mean,5.494973e+17,0.003197,0.004064,1.354248,-0.998305,5.053305,1.350184,4.242442e+06
std,2.597497e+17,0.056451,0.076035,0.572840,1.455093,2.141806,0.570082,3.351295e+06
min,1.011555e+17,0.000000,0.000000,1.000000,-2.766667,0.933333,0.000000,1.000000e+06
25%,3.239523e+17,0.000000,0.000000,1.000000,-1.650000,4.066667,1.000000,2.100000e+06
50%,5.452523e+17,0.000000,0.000000,1.000000,-1.033333,4.100000,1.000000,2.900000e+06
75%,7.752523e+17,0.000000,0.000000,2.000000,-0.833333,5.100000,2.000000,5.200000e+06
max,9.999553e+17,1.000000,2.000000,4.000000,8.566667,14.166667,4.000000,2.000000e+07


In [32]:
tabla_final.dtypes

id_cliente                     int64
mes_campaña                   object
acep_oferta_prev             float64
mes_ult_act           datetime64[ns]
mes_ult_camp          datetime64[ns]
num_act_utl_meses            float64
num_camp_ult_meses           float64
num_meses_ult_actv           float64
num_meses_ult_camp           float64
num_no_aceptado              float64
prom_cupo_aprob              float64
dtype: object

In [33]:
tabla_final = tabla_final.drop_duplicates()

In [34]:
tabla_final['data_camp'] = pd.to_datetime(tabla_final['mes_campaña'], format='%Y-%m-%d').dt.strftime('%Y%m').astype('int')
tabla_final

,id_cliente,mes_campaña,acep_oferta_prev,mes_ult_act,mes_ult_camp,num_act_utl_meses,num_camp_ult_meses,num_meses_ult_actv,num_meses_ult_camp,num_no_aceptado,prom_cupo_aprob,data_camp
0,605152297523531501,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,201902
1,309252297762338201,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,201902
2,365655266241757701,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,201902
3,227552315277170601,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,201902
4,833752296410065002,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,201902
...,...,...,...,...,...,...,...,...,...,...,...,...
88964,132452296355094801,2020-03-01,0.0,NaT,2019-11-01,0.0,3.0,NaN,4.033333,3.0,19600000.0,202003
88965,246955266243335502,2020-03-01,0.0,NaT,2019-11-01,0.0,2.0,NaN,4.033333,2.0,12000000.0,202003
88966,102652305802784101,2020-03-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,202003
88967,339352294618484201,2020-03-01,0.0,NaT,2019-09-01,0.0,1.0,NaN,6.066667,1.0,NaN,202003


In [35]:
tabla_final.shape

(88969, 12)

In [36]:
tabla_final.count()

id_cliente            88969
mes_campaña           88969
acep_oferta_prev      18456
mes_ult_act              59
mes_ult_camp          18456
num_act_utl_meses     18456
num_camp_ult_meses    18456
num_meses_ult_actv       59
num_meses_ult_camp    18456
num_no_aceptado       18456
prom_cupo_aprob        7535
data_camp             88969
dtype: int64

In [37]:
base_final[base_final['ID_CLIENTE']=='971552297263756702']

,CUPO_APROBADO,FECHA,ID_CLIENTE,Numero_cuenta,TIPO CAMPAÑA,camp_201901,camp_201902,camp_201903,camp_201904,camp_201905,camp_201906,camp_201907,camp_201908,camp_201909,camp_201910,camp_201911,camp_201912,camp_202001,camp_202002,camp_202003,cedula,fecha_activacion,mes_activacion,mes_campaña,tipo_camp,venta
1444,NaN,2019-02-26,971552297263756702,NaN,Preaprobados,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23132562,NaT,NaT,2019-02-01,of-preaprobados,0.0


In [38]:
tabla_final[tabla_final['id_cliente']==999756158932840302]

,id_cliente,mes_campaña,acep_oferta_prev,mes_ult_act,mes_ult_camp,num_act_utl_meses,num_camp_ult_meses,num_meses_ult_actv,num_meses_ult_camp,num_no_aceptado,prom_cupo_aprob,data_camp
15077,999756158932840302,2019-04-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,201904
33715,999756158932840302,2019-08-01,0.0,NaT,2019-04-01,0.0,1.0,NaN,4.066667,1.0,2400000.0,201908
61546,999756158932840302,2019-12-01,0.0,NaT,2019-08-01,0.0,2.0,NaN,4.066667,2.0,2400000.0,201912


### Guardando Base:

In [39]:
path_final = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/var_camp_act_2019'

tabla_final.to_parquet(path_final, engine = 'pyarrow', index = False)